In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"]= (10,2)
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings("ignore")
import plotly.express as px

In [3]:
""" data ingestion """

lst = ['wm',"ac1",'ac2',"f",'tv']
path = "d:/NILM/Data_files/Parquet/Appliances12_06-04_08/"

""" data ingestion done """

""" data validation and transformation """
complete_data_lst = []
date_dit ={}
for i in range(len(lst)):
    counter = 0
    dit = {}
    df = pd.read_parquet(f"{path}{lst[i]}.parquet")
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.drop_duplicates(inplace=True)
    df.set_index(['timestamp'],inplace=True ,drop=True)  
    df.drop(df[(df['frequency']>51) | (df['frequency']<49)].index, inplace=True)
    df.drop(df[(df['PF']>1) | (df['PF']<0)].index, inplace=True)
    # df.drop(df[(df['current']>20) |(df['current']<0.07)].index, inplace=True)
    df.drop(df[df['voltage']<140].index, inplace=True)
    
    """ data validation done """
    
    """ data transformation """
    df['power'] = (df['voltage'] * df['current'] * df['PF']).round(2)
    if lst[i]=="f":
        df_appliance = df[['power',"voltage",'current']].resample(rule="1s").asfreq()
        df_appliance = df_appliance.rename(columns={'power': f'{lst[i]}_A', 'voltage': f'{lst[i]}_V', 'current': f'{lst[i]}_C'})
        unique_dates = pd.Series(df.index.date).unique()
        for date1 in unique_dates:
            hours_lst = []
            df_date = df[df.index.to_series().dt.date == date1]
            hours = df_date.index.hour.unique()
            for y in hours:
                hours_lst.append(y)
                counter+=1
            dit[f"{date1}"] = hours_lst
    else:
        df_sample_lst = []
        unique_dates = pd.Series(df.index.date).unique()
        for date1 in unique_dates:
            hours_lst = []
            df_date = df[df.index.to_series().dt.date == date1]
            hours = df_date.index.hour.unique()
            for y in hours:
                hours_lst.append(y)
                date2 = f"{date1} {y:02d}"
                df_hour = df_date.loc[date2]
                df_resampled_hour = df_hour[['power',"voltage",'current']].resample(rule="1s").asfreq()
                df_sample_lst.append(df_resampled_hour)
                counter+=1
            
            dit[f"{date1}"] = hours_lst

        df_appliance = pd.concat(df_sample_lst)
        df_appliance = df_appliance.rename(columns={'power': f'{lst[i]}_A', 'voltage': f'{lst[i]}_V', 'current': f'{lst[i]}_C'})

    date_dit[f"{lst[i]}"] = dit
    print(lst[i])
    print("days when appliance is active :", len(unique_dates))
    print("no. of hours:", counter)
    complete_data_lst.append(df_appliance)


wm
days when appliance is active : 14
no. of hours: 26
ac1
days when appliance is active : 50
no. of hours: 148
ac2
days when appliance is active : 46
no. of hours: 207
f
days when appliance is active : 54
no. of hours: 1258
tv
days when appliance is active : 37
no. of hours: 285


In [4]:
for i in range(len(complete_data_lst)):
    counter = 0
    df = complete_data_lst[i]
    unique_dates = pd.Series(df.index.date).unique()
    for date1 in unique_dates:
        df_date = df[df.index.to_series().dt.date == date1]
        hours = df_date.index.hour.unique()
        for y in hours:
            counter+=1
    print("days when appliance is active :", len(unique_dates))
    print("no. of hours:", counter)

days when appliance is active : 14
no. of hours: 26
days when appliance is active : 50
no. of hours: 148
days when appliance is active : 46
no. of hours: 207
days when appliance is active : 54
no. of hours: 1273
days when appliance is active : 37
no. of hours: 285


In [5]:
for i in range(len(complete_data_lst)):
    df = complete_data_lst[i]
    unique_dates = pd.Series(df.index.date).unique()
    print(len(unique_dates))

14
50
46
54
37


In [12]:
date_dit.keys()

dict_keys(['wm', 'ac1', 'ac2', 'f', 'tv'])

In [16]:
date_dit['wm']

{'2013-06-14': [1],
 '2013-06-17': [4],
 '2013-06-21': [0, 1],
 '2013-06-24': [5],
 '2013-06-28': [1, 2],
 '2013-07-02': [5, 6],
 '2013-07-07': [1],
 '2013-07-10': [1, 2, 4, 12],
 '2013-07-14': [1, 2],
 '2013-07-17': [1, 2, 4],
 '2013-07-21': [1, 3],
 '2013-07-25': [2],
 '2013-07-28': [1, 2],
 '2013-07-31': [1, 2]}